In [1]:
import pyanitools as pyt
import pyaniasetools as aat
import hdnntools as hdt
import numpy as np

import matplotlib.pyplot as plt
import aniensemblestats as ens

import os
import pandas as pd

In [2]:
fdir = '/home/jujuman/Research/extensibility_test_sets/ani_md_benchmark/data_big/data/'

wkdir = '/home/jujuman/Research/DataReductionMethods/model6r/model-gdb_r06_comb09_1/cv4/'
#kdir = '/home/jujuman/Research/ANI-validation/'
cnstfile = wkdir + 'rHCNO-4.6A_16-3.1A_a4-8.params'
saefile  = wkdir + 'sae_6-31gd.dat'
nnfdir   = wkdir + '/train'
Nn = 5

In [3]:
# Define the conformer cross validator class
anicv = aat.anicrossvalidationconformer(cnstfile,saefile,nnfdir,Nn,0,False)

In [4]:
files = os.listdir(fdir)

stats = dict()
for f in files:
    name = f.split('_')[0]
    data = hdt.readncdatall(fdir+f)
    
    Eani, Fani = anicv.compute_energy_conformations(X=data['coordinates'],S=data['species'])
    
    Eani = np.mean(Eani,axis=0)
    Fani = -np.mean(Fani,axis=0)
    
    Edft = hdt.hatokcal*data['energies']
    Fdft = hdt.hatokcal*data['forces']
    
    #print(name)
    #print(data['species'])
    #print((Fani-Fdft)[0][12])
    #print(Fani[0][12])
    #print(Fdft[0][12])

    Fani = Fani.flatten()
    Fdft = Fdft.flatten()
    
    dEani = hdt.calculatedmat(Eani)
    dEdft = hdt.calculatedmat(Edft)
    
    Na = len(data['species'])
    
    stats[name] = pd.Series({'# Atoms': int(Na),
                             'EMAE'   : hdt.calculatemeanabserror(Eani,Edft),
                             'ERMS'   : hdt.calculaterootmeansqrerror(Eani,Edft),
                             'ERNG'   : np.abs(Edft.min()-Edft.max()),
                             'dEMAE'  : hdt.calculatemeanabserror(dEani,dEdft),
                             'dERMS'  : hdt.calculaterootmeansqrerror(dEani,dEdft),
                             'dERNG'  : np.abs(dEdft.min()-dEdft.max()),
                             'FMAE'   : hdt.calculatemeanabserror(Fani,Fdft),
                             'FRMS'   : hdt.calculaterootmeansqrerror(Fani,Fdft),
                             'FRNG'   : np.abs(Fdft.min()-Fdft.max())
                            })
    
edat = pd.DataFrame(stats).transpose()

In [5]:
print(edat.sort_values(['# Atoms']))

           # Atoms  EMAE  ERMS  ERNG  FMAE  FRMS   FRNG  dEMAE  dERMS  dERNG
Capsaicin    49.00  1.62  2.13 25.79  2.06  2.95 231.13   2.08   2.66  50.26
